# OCR Demystified !!
## [ Part-2 ]  Training Data and Feature extraction

### Feature Extraction

These features are extracted for each character

1. Width
2. Height
3. Aspect Ratio
4. Number of pixels
5. Ratio of white to black pixels
6. Horizontal symmetry
7. Vertical symmetry
8. x histogram
9. y histogram


In [3]:
#Importing general libraries

import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
import numpy as np
import os

from PIL import Image, ImageChops, ImageDraw, ImageFilter, ImageOps


In [31]:
# functions for reading and showing images

def readImage(index):
    image=Image.open(os.getcwd()+'/Images/'+str(index)+'.jpg')
    return image

def showImage(image):
    plt.imshow(image)
    plt.show()

In [32]:
# Getting the max width and height of training images generated in previous step
%store -r max_values
max_width, max_height = max_values
print 'max_width',max_width
print 'max_height',max_height

max_width 19
max_height 33


In [36]:
# Creating final training images and calling extractFeatures()

X_train = []
Y_train = []
X_test = []

def generateData(task="train"):
    
    if(task=="train"):
        global X_train, Y_train
        X_train = []
        Y_train = []
        inDir = os.getcwd()+'/characters/'
        outDir = './training_data/' 
        
    else:
        global X_test
        X_test = []
        inDir = os.getcwd()+'/test_characters/'
        outDir = './test_data/'
        
    images = os.listdir(inDir)
    for imageName in images:

        #Creating a blank image of size max_width * max_height
        background = Image.new("1", (max_width, max_height), "white")

        image = Image.open(inDir+imageName)
        width, height = image.size

        background.paste(image, ((max_width/2)-(width/2), (max_height/2)-(height/2)))
        background.save(outDir + imageName)

        extractFeature(background, image, imageName, task)

    if(task == "train"):
        print 'Stats: TRAINING\n'
        print 'Size of training data: ', len(X_train),'\n'
        print 'Number of features: ', len(X_train[0]),'\n'

        print 'A data point in final training data: \n', X_train[0], '\n'
        print 'A label in training data: \n',Y_train[0], '\n'

        shape = list(X_train[0].shape)
        shape[:0] = [len(X_train)]
        X_train = np.concatenate( X_train).reshape(shape)

        #Saving training data on disk
        np.save("X_train", X_train)
        print "Saved X_train"
        np.save("Y_train", Y_train)
        print "Saved Y_train"
        
    else:
        print 'Stats: TESTING\n'
        print 'Size of testing data: ', len(X_test),'\n'
        print 'Number of features: ', len(X_test[0]),'\n'
        print 'A data point in final training data: \n', X_test[0], '\n'
        
        shape = list(X_test[0].shape)
        shape[:0] = [len(X_test)]
        X_test = np.concatenate( X_test).reshape(shape)
        
        np.save("X_test", X_test)
        print "Saved X_test"
    
    
generateData("test")        



Stats: TESTING

Size of testing data:  6 

Number of features:  57 

A data point in final training data: 
[  12   24  288 2350 2266   33   33   33   30   30   29   10    9    9   12
   28   28   28   29   31   33   33   33   33   19   19   19   19   15   15
   15   14    7    7    8   15   15   15   15   15   15   15   16   15   16
   15   16   15   15   15   11   13   19   19   19   19   19] 

Saved X_test


In [19]:
def extractFeature(background, image, imageName, task):
    #Extracts features from characters
    
    xtrain = []
    
    #feature 1: width
    xtrain.append(image.size[0])
    
    #feature 2: height
    xtrain.append(image.size[1])
    
    #feature : aspect ratio
#     xtrain.append(float(image.size[1])/image.size[0])
    
    #feature : number of pixels
    xtrain.append(image.size[1]*image.size[0])
    
    
    #feature 3: ratio of white to black pixels
    whitePixels = white_pixels(background)
    totalPixels = background.size[0]*background.size[1]
    blackPixels = totalPixels - whitePixels
#     xtrain.append(whitePixels/blackPixels)
    
    #feature 3: vertical_symmetry
    value = vertical_symmetry(background)
    xtrain.append(value)
    
    #feature 3: horizontal_symmetry
    value = horizontal_symmetry(background)
    xtrain.append(value)
    
    #featuer 3: x_histogram
    x_hist_values = x_histogram(background)
    xtrain = np.concatenate((xtrain, x_hist_values), axis = 0)
    
    #featuer 4: y_histogram
    y_hist_values = y_histogram(background)
    xtrain = np.concatenate((xtrain, y_hist_values), axis = 0)
    
    
    #Writing to final data
    if(task=="train"):
        X_train.append(xtrain)
        Y_train.append(imageName.split("_")[2].split(".")[0])
    else:
        X_test.append(xtrain)
    
    

In [8]:
def x_histogram(background):
    hist_values = []
    width,height = background.size
    pix = np.asarray(background).transpose()
    for x in range(width):
        hist_values.append( np.add.reduce(pix[x]))
    return hist_values

def y_histogram(background):
    hist_values = []
    width,height = background.size
    pix = np.asarray(background)
    for x in range(height):
        hist_values.append( np.add.reduce(pix[x]))
    return hist_values

def white_pixels(background):
    pixeldata = np.asarray(background)
    return sum(np.add.reduce(pixeldata))

def vertical_symmetry(background):
    width,height = background.size
    first_half = np.array(background.crop((0, 0, width/2, height)).getdata())
    second_half = np.array(ImageOps.mirror(background.crop((width/2, 0, width, height)).getdata()))
    second_half = second_half[:len(first_half)]
    return int(np.linalg.norm(first_half-second_half))

def horizontal_symmetry(background):
    width,height = background.size
    first_half = np.array(background.crop((0, 0, width, height/2)).getdata())
    second_half = np.array(ImageOps.mirror(background.crop((0, height/2, width, height)).getdata()))
    second_half = second_half[:len(first_half)]
    return  int(np.linalg.norm(first_half-second_half))
    
